# Instalando dependências

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

In [ ]:
if IN_COLAB:
    !git clone https://github.com/LucaLemos/UFRPE_AprendizagemReforco
    sys.path.append("/content/UFRPE_AprendizagemReforco")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

In [ ]:
if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg
    !pip install gymnasium==1.0.0   # conferir se precisa
    #!pip install tianshou # Para criar o Replay_Buffer
    #!pip install d3rlpy==2.7.0
    # clone repository

# Criando Dataset

In [1]:
import gymnasium as gym
import torch
from util.algorithms import run_sarsa
from util.network import ReplayBuffer, MLP, epsilon_greedy_qnet, calc_loss
from IPython.display import clear_output

In [2]:
DATASET_SIZE = 200_000  # Tamanho do conjunto de dados (replay buffer)
LEARNING_RATE = 1e-3  # Taxa de aprendizado para o otimizador
GAMMA = 0.99  # Fator de desconto
BATCH_SIZE = 128  # Tamanho do batch para treinamento da rede neural

In [3]:
ENV_NAMES = []
ENVS_REPLAY_BUFFER = []

## Dataset Básico

In [4]:
# Passo 1: Coletar um conjunto fixo de transições (Replay Buffer)
# Ambientes Básicos
ENV_NAMES += ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0"]
for i, env_name in enumerate(ENV_NAMES):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    replay_buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    env = gym.make(env_name, render_mode="rgb_array")
    
    sum_rewards_per_ep, q = run_sarsa(env, replay_buffer, DATASET_SIZE, LEARNING_RATE, GAMMA)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer, sum_rewards_per_ep, q))
    replay_buffer.save_config(f"config\dataset\sarsa\{env_name}.json")


Run sarsa in <TimeLimit<OrderEnforcing<PassiveEnvChecker<FrozenLakeEnv<FrozenLake-v1>>>>>
Episódio 0 terminou com recompensa 0.0 na transição 3
Episódio 100 terminou com recompensa 0.0 na transição 1601
Episódio 200 terminou com recompensa 0.0 na transição 3020
Episódio 300 terminou com recompensa 0.0 na transição 4652
Episódio 400 terminou com recompensa 0.0 na transição 6299
Episódio 500 terminou com recompensa 0.0 na transição 7809
Episódio 600 terminou com recompensa 0.0 na transição 9291
Episódio 700 terminou com recompensa 0.0 na transição 10720
Episódio 800 terminou com recompensa 0.0 na transição 12090
Episódio 900 terminou com recompensa 0.0 na transição 13643
Episódio 1000 terminou com recompensa 0.0 na transição 15112
Episódio 1100 terminou com recompensa 0.0 na transição 16545
Episódio 1200 terminou com recompensa 0.0 na transição 18010
Episódio 1300 terminou com recompensa 0.0 na transição 19488
Episódio 1400 terminou com recompensa 0.0 na transição 21061
Episódio 1500 ter

## Dataset com Rede Neural

In [ ]:
from torch import optim, nn
import numpy as np

In [ ]:
def DQN_TRAIN(env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_buffer, batch_size, epsilon_f, epsilon_decay_period, transitions):
    # Cria o otimizador, que vai fazer o ajuste dos pesos da 'qnet',
    # Usa uma técnica de gradiente descendente de destaque, chamada ADAM
    optimizer = optim.Adam(qnet.parameters(), lr=qnet_lr)

    episode_reward_list = []
    step = 0
    epsilon = 1.0

    state, _ = env.reset()
    episode_reward = 0.0

    for _ in range(transitions):
        # Decaimento linear do epsilon
        epsilon = max(epsilon_f, 1.0 - step / epsilon_decay_period)

        action = epsilon_greedy_qnet(qnet, env, state, epsilon)

        # Faz um passo / Aplica uma ação no ambiente
        new_state, reward, terminated, truncated, _ = env.step(action)

        step += 1
        done = terminated or truncated
        episode_reward += reward

        # Adiciona no buffer
        replay_buffer.add(state, action, reward, terminated, new_state)
        state = new_state

        if done:
            episode_reward_list.append(episode_reward)
            if len(episode_reward_list) % 100 == 0:
                mean_reward = np.mean(episode_reward_list[-100:])
                print(f'Episode Reward: {episode_reward_list[-1]} | Epsilon: {epsilon} | Step: {step} | Mean Reward: {mean_reward}')
            state, _ = env.reset()
            episode_reward = 0.0

            # Abaixo, faz vários loggings de dados
            
        # Faz a 'tgt_net' receber os mesmos valores de pesos da 'qnet', na frequência indicada
        
        if step > batch_size:
            if step % target_update_freq == 0:
                target_qnet.load_state_dict(qnet.state_dict())
            # Escolhe amostras aleatórios do buffer e faz uma atualização dos pesos da rede
            optimizer.zero_grad()
            batch = replay_buffer.sample_continuous()
            loss_t = calc_loss(batch, qnet, target_qnet, gamma)
            loss_t.backward()
            optimizer.step()
            
    mean_reward = np.mean(episode_reward_list[-100:])
    print(f"Mean_reward: {mean_reward}")
    filename = f"./neural_dataset/{env_name}.pth"
    torch.save(qnet.state_dict(), filename)
    print(f"Model saved as {filename}")

In [ ]:
GOAL_REWARD = 200
GAMMA = 0.999
REPLAY_SIZE = 100_000
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
SYNC_TARGET_FRAMES = 1_000

EPSILON_DECAY_PERIOD = 75_000
EPSILON_FINAL = 2e-2

In [ ]:
#ENV_NAMES += ["CartPole-v1"]
ENV_NAMES += ["LunarLander-v3"]
for i, env_name in enumerate(ENV_NAMES[-1:]):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #replay_buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    buffer = ReplayBuffer(DATASET_SIZE, BATCH_SIZE, device)
    print(env_name)
    env = gym.make(env_name)
    
    qnet1 = MLP(env.observation_space.shape[0], env.action_space.n, [256,256])
    qtarget1 = MLP(env.observation_space.shape[0], env.action_space.n, [256,256])

    DQN_TRAIN(
    env = env,
    env_name = env_name,
    gamma = GAMMA,
    qnet = qnet1,
    qnet_lr = LEARNING_RATE,
    target_qnet = qtarget1,
    target_update_freq = SYNC_TARGET_FRAMES,
    replay_buffer= buffer,
    batch_size = BATCH_SIZE,
    epsilon_f = EPSILON_FINAL,
    epsilon_decay_period = EPSILON_DECAY_PERIOD,
    transitions = REPLAY_SIZE)

    #!sum_rewards_per_ep, q = run_sarsa(env, replay_buffer, DATASET_SIZE, LEARNING_RATE, GAMMA)
    ENVS_REPLAY_BUFFER.append((env_name, env, buffer))
    buffer.save_config(f"config\dataset\dqn\{env_name}.json")

# Treinando o Modelo

## Carregando Datasets

In [ ]:
import gymnasium as gym
from util.network import ReplayBuffer


In [ ]:
ENV_NAMES = ["FrozenLake-v1", "Taxi-v3", "CliffWalking-v0", "CartPole-v1", "LunarLander-v3"]

In [ ]:
ENVS_REPLAY_BUFFER = []
for env_name in ENV_NAMES[:3]:
    replay_buffer = ReplayBuffer.load_config(f"config\dataset\sarsa\{env_name}.json")
    env = gym.make(env_name)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer))
for env_name in ENV_NAMES[3:]:
    replay_buffer = ReplayBuffer.load_config(f"config\dataset\dqn\{env_name}.json")
    env = gym.make(env_name)
    ENVS_REPLAY_BUFFER.append((env_name, env, replay_buffer))

## Definições de Treino

In [ ]:
import argparse

In [ ]:
def get_config(env_name, count_episodes, seed, gamma, tau, lr, steps, hidden_size, isDiscrete):
    parser = argparse.ArgumentParser(description='RL')
    parser.add_argument("--run_name", type=str, default=f"{env_name}-DQN-FQI", help="Run name, default: FQI-DQN")
    parser.add_argument("--env", type=str, default=env_name, help="Gym environment name, default: CartPole-v0")
    parser.add_argument("--episodes", type=int, default=count_episodes, help="Number of episodes, default: 200")
    parser.add_argument("--seed", type=int, default=seed, help="Seed, default: 1")
    parser.add_argument("--steps", type=int, default=steps, help="Saves the network every x epochs, default: 25")
    
    parser.add_argument("--gamma", type=float, default=gamma, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--tau", type=float, default=tau, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--lr", type=float, default=lr, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--hidden_size", type=list[int], default=hidden_size, help="Saves the network every x epochs, default: 25")
    parser.add_argument("--isDiscrete", type=bool, default=isDiscrete, help="Saves the network every x epochs, default: 25")
    
    args, _ = parser.parse_known_args()
    return args

In [ ]:
from util.network import FQIAgent, to_one_hot, MLP
import numpy as np
import random
import torch
from collections import deque

In [ ]:
def train_DQN_FQI(config, buffer):
    np.random.seed(config.seed)
    random.seed(config.seed)
    torch.manual_seed(config.seed)
    env = gym.make(config.env)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    losses = deque(maxlen=1000)
    if config.isDiscrete:
        agent = FQIAgent(env.observation_space.n, env.action_space.n, config.tau, config.gamma, config.lr, config.hidden_size, device, config.isDiscrete)
    else:
        agent = FQIAgent(env.observation_space.shape[0], env.action_space.n, config.tau, config.gamma, config.lr, config.hidden_size, device, config.isDiscrete)
    for i in range(config.steps):
        loss = agent.learn(buffer.sample_continuous())
        losses.append(loss)     
        if i % 1000 == 0:
            print(f"[TRAIN {i}] Mean Q Loss: {np.mean(losses)} ")
    
    agent.save(config, save_name="FQI-DQN")
    
    return env

In [ ]:
def evaluate(config, env, model):
    total_rewards = []
    num_episodes = config.episodes
    for i in range(num_episodes):
        print(f"[Episódio {i}]")
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step = 0
        while not done:
            state_tensor = torch.tensor([state], dtype=torch.long)
            state_tensor = to_one_hot(model.state_size, state_tensor)[0].float().unsqueeze(0)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()
            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1
            if step == 10000:
                break
        total_rewards.append(episode_reward)

    print(f"Média de recompensa após {num_episodes} episódios: {sum(total_rewards) / num_episodes}")

In [ ]:
# Função para extrair a tabela Q da rede neural
def extract_q_table(q_network, num_states, num_actions):
    q_table = np.zeros((num_states, num_actions))
    for state in range(num_states):
        state_tensor = torch.tensor([state], dtype=torch.long)
        state_tensor = to_one_hot(num_states, state_tensor)[0].float().unsqueeze(0)
        q_values = q_network(state_tensor).detach().numpy()
        q_table[state] = q_values
    return q_table

In [ ]:
from util.qtable_helper import record_video_qtable
from util.notebook import display_videos_from_path

In [ ]:

def extract_and_display(model, env_name):
    # Extrair a tabela Q da rede neural treinada
    q_table = extract_q_table(model, model.state_size, model.action_size)

    # Gravar um vídeo da política treinada
    video_path = 'videos/'  # Pasta onde os vídeos serão salvos
    video_prefix = f"fqi_{env_name}"  # Prefixo para o nome do arquivo de vídeo

    # Gravar o vídeo
    record_video_qtable(env_name, q_table, episodes=2, folder=video_path, prefix=video_prefix)

    # Exibir o vídeo gravado
    display_videos_from_path(video_path, prefix=video_prefix)

In [ ]:
def evaluate_Continuous(config, env, model):
    total_rewards = []
    num_episodes = config.episodes
    for i in range(num_episodes):
        print(f"[Episódio {i}]")
        state, _ = env.reset()
        done = False
        episode_reward = 0.0
        step = 0
        while not done:
            state_v = torch.tensor(state, dtype=torch.float32)
            state_v = state_v.unsqueeze(0)  # Adiciona dimensão de batch como eixo 0 (e.g. transforma uma lista [a,b,c] em [[a,b,c]])
            with torch.no_grad():
                q_vals_v = model(state_v)
                _, act_v = torch.max(q_vals_v, dim=1)
            action = int(act_v.item())


            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1
            if step == 10000:
                break
        total_rewards.append(episode_reward)

    print(f"Média de recompensa após {num_episodes} episódios: {sum(total_rewards) / num_episodes}")

In [ ]:
# Função para renderizar o ambiente com o modelo treinado
import time
def render_environment(model, env_name, device="cpu", episodes=5, max_steps=100000):
    env = gym.make(env_name, render_mode="human")  # Modo "human" para renderização
    for episode in range(episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0
        step = 0

        while not done and step < max_steps:
            # Renderiza o ambiente
            env.render()
            time.sleep(0.02)  # Adiciona um pequeno delay para visualização

            # Converte o estado para um tensor e passa pelo modelo
            state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
            with torch.no_grad():
                q_values = model(state_tensor)
                action = torch.argmax(q_values).item()

            # Executa a ação no ambiente
            state, reward, done, truncated, info = env.step(action)
            episode_reward += reward
            step += 1

            if done or truncated:
                print(f"Episódio {episode + 1}: Recompensa = {episode_reward}, Passos = {step}")
                break

    env.close()

## FrozenLake

In [ ]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 32
STEPS = 100_000
GAMMA = 0.95 # Fator de desconto
TAU = 1e-1      # Taxa de atualização da target_network

LEARNING_RATE = 3e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [64, 32]
ISDISCRETE = True
# Extra:

In [ ]:
ENVS_REPLAY_BUFFER[0][2].batch_size = BATCH_SIZE

In [ ]:
config = get_config(ENVS_REPLAY_BUFFER[0][0], COUNT_EPISODES, SEED, GAMMA, TAU, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [ ]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[0][2])

In [ ]:
env = ENVS_REPLAY_BUFFER[0][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[0][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

In [ ]:
evaluate(config, env, model)

In [ ]:
extract_and_display(model, ENVS_REPLAY_BUFFER[0][0])

## Taxi

In [ ]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 300_000
GAMMA = 0.97  # Fator de desconto
TAU = 1e-2      # Taxa de atualização da target_network
LEARNING_RATE = 5e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [512, 256]
ISDISCRETE = True
# Extra:

In [ ]:
ENVS_REPLAY_BUFFER[1][2].batch_size = BATCH_SIZE

In [ ]:
config = get_config(ENVS_REPLAY_BUFFER[1][0], COUNT_EPISODES, SEED, GAMMA, TAU, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [ ]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[1][2])

In [ ]:
env = ENVS_REPLAY_BUFFER[1][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[1][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

In [ ]:
evaluate(config, env, model)

In [ ]:
extract_and_display(model, ENVS_REPLAY_BUFFER[1][0])

## Cliffwalking

In [ ]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 128
STEPS = 200_000
GAMMA = 0.98  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
LEARNING_RATE = 5e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 128, 64]  # Tamanho da camada oculta
ISDISCRETE = True
# Extra:

In [ ]:
ENVS_REPLAY_BUFFER[2][2].batch_size = BATCH_SIZE

In [ ]:
config = get_config(ENVS_REPLAY_BUFFER[2][0], COUNT_EPISODES, SEED, GAMMA, TAU, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [ ]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[2][2])

In [ ]:
env = ENVS_REPLAY_BUFFER[2][1]
model = MLP(env.observation_space.n, env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[2][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

In [ ]:
evaluate(config, env, model)

In [ ]:
extract_and_display(model, ENVS_REPLAY_BUFFER[2][0])

## CartPole

In [ ]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 50_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
LEARNING_RATE = 1e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 128]
ISDISCRETE = False
# Extra:

In [ ]:
ENVS_REPLAY_BUFFER[3][2].batch_size = BATCH_SIZE

In [ ]:
config = get_config(ENVS_REPLAY_BUFFER[3][0], COUNT_EPISODES, SEED, GAMMA, TAU, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [ ]:
ENVS_REPLAY_BUFFER[3][2].memory

In [ ]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[3][2])

In [ ]:
env = ENVS_REPLAY_BUFFER[3][1]
model = MLP(env.observation_space.shape[0], env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[3][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

In [ ]:
evaluate_Continuous(config, env, model)

In [ ]:
#extract_and_display(model, ENVS_REPLAY_BUFFER[3][0])
render_environment(model, ENVS_REPLAY_BUFFER[3][0])

## LunarLander

In [ ]:
COUNT_EPISODES = 50
SEED = 777
BATCH_SIZE = 256
STEPS = 50_000
GAMMA = 0.99  # Fator de desconto
TAU = 5e-3      # Taxa de atualização da target_network
LEARNING_RATE = 1e-4  # Taxa de aprendizado para o otimizador
HIDDEN_SIZE = [256, 256]
ISDISCRETE = False
# Extra:

In [ ]:
ENVS_REPLAY_BUFFER[4][2].batch_size = BATCH_SIZE

In [ ]:
config = get_config(ENVS_REPLAY_BUFFER[4][0], COUNT_EPISODES, SEED, GAMMA, TAU, LEARNING_RATE, STEPS, HIDDEN_SIZE, ISDISCRETE)

In [ ]:
env = train_DQN_FQI(config, ENVS_REPLAY_BUFFER[4][2])

In [ ]:
env = ENVS_REPLAY_BUFFER[4][1]
model = MLP(env.observation_space.shape[0], env.action_space.n, HIDDEN_SIZE)
model.load_state_dict(torch.load(f"trained_models/{ENVS_REPLAY_BUFFER[4][0]}FQI-DQN.pth"))
model.eval()  # Definir para modo de avaliação

In [ ]:
evaluate_Continuous(config, env, model)

In [ ]:
#extract_and_display(model, ENVS_REPLAY_BUFFER[3][0])
render_environment(model, ENVS_REPLAY_BUFFER[4][0])